In [1]:
!pip install pyspark

     |████████████████████████████████| 311.4 MB 39 kB/s s eta 0:00:01
     |████████████████████████████████| 200 kB 33.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.4.4-py2.py3-none-any.whl size=311905484 sha256=d12b0dc843e2c99423cd34802ab392a021369e35de5fe119c2359ca917277139
  Stored in directory: /root/.cache/pip/wheels/4e/66/db/939eb1c49afb8a7fd2c4e393ad34e12b77db67bb4cc974c00e
Successfully built pyspark


In [2]:
#load_ext nb_black
from pyspark.sql import SparkSession
#session all related to df not rdd
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/12 16:08:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [3]:
spark

In [4]:
#import os
#from os.path import isfile, join

#loc = os.path.abspath("")
#data_loc = f"{loc}/data/"

Dataset: https://www.kaggle.com/ntnu-testimon/paysim1/data (Randomly sampled 10% of the dataset)

In [5]:
df = spark.read.csv("../input/paysim1", inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



In [7]:
df.show(2)

+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|   type| amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1|PAYMENT|9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1|PAYMENT|1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
only showing top 2 rows



Drop columns from the data :
* In any machine learning project, we always have a few columns that are not required for solving the problem.

for instance, we can use the drop function to remove the column from the data. Use the asterisk (*) sign before the list to drop multiple columns from the dataset:
my_data = my_data.drop(*['Batsman', 'Bowler', 'Id'])
my_data.columns

For our instance ,we don't really need " nameOrig " , " nameDest " , " oldbalanceDest " , " oldbalanceDest " ," newbalanceDest " ,we really don't need these as these are not the factors which affects the frud data or not.

In [8]:
df = df.select("type", "amount", "oldbalanceOrg", "newbalanceOrig", "isFraud")

we are only slecting some colum which is an alternative for droping a colum rather ,efficent.

In [9]:
df.show(2)

+-------+-------+-------------+--------------+-------+
|   type| amount|oldbalanceOrg|newbalanceOrig|isFraud|
+-------+-------+-------------+--------------+-------+
|PAYMENT|9839.64|     170136.0|     160296.36|      0|
|PAYMENT|1864.28|      21249.0|      19384.72|      0|
+-------+-------+-------------+--------------+-------+
only showing top 2 rows



In [10]:
df.printSchema()

root
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- isFraud: integer (nullable = true)



## Data Dimensions 

In [11]:
(df.count() , len(df.columns))

(6362620, 5)

In [12]:
df.select('amount','oldbalanceOrg','newbalanceOrig','isFraud').describe().show()

24/12/12 16:09:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+-----------------+--------------------+
|summary|            amount|    oldbalanceOrg|   newbalanceOrig|             isFraud|
+-------+------------------+-----------------+-----------------+--------------------+
|  count|           6362620|          6362620|          6362620|             6362620|
|   mean|179861.90354913412|833883.1040744719|855113.6685785714|0.001290820448180152|
| stddev| 603858.2314629498|2888242.673037545|2924048.502954253|0.035904796801604424|
|    min|               0.0|              0.0|              0.0|                   0|
|    max|     9.244551664E7|    5.958504037E7|    4.958504037E7|                   1|
+-------+------------------+-----------------+-----------------+--------------------+



In [13]:
# null values in each column
data_agg = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])
data_agg.show()

+----+------+-------------+--------------+-------+
|type|amount|oldbalanceOrg|newbalanceOrig|isFraud|
+----+------+-------------+--------------+-------+
|   0|     0|            0|             0|      0|
+----+------+-------------+--------------+-------+



In [14]:
# value counts of Type column
df.groupBy('type').count().show()

+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 532909|
| CASH_IN|1399284|
|CASH_OUT|2237500|
| PAYMENT|2151495|
|   DEBIT|  41432|
+--------+-------+



### Train/test split

In [15]:
train, test = df.randomSplit([0.7, 0.3], seed=7)

so here is thr things we need to select data for train and test 


random selection of data makes the good distribution of record ,leads to good model

data weight is in fraction so 
0.7 ie 70% of the data from is being randomly selected and loaded but with a random factor seed ie 7

In [16]:
#it better be random so that we will have a better distribution and so better model.


randomSplit:
Seed is random number generator seed. This is important because you might want to be able to hard code the same seed for your tests so that you always get the same results in test, but in prod code replace it with current time in milliseconds or a random number from a good entropy source.

Basically the splited in random way. 

In [17]:
print(f"Train set length: {train.count()} records")
print(f"Test set length: {test.count()} records")

Train set length: 4451877 records


Test set length: 1910743 records


In [18]:
train.show(2)

+-------+------+-------------+--------------+-------+
|   type|amount|oldbalanceOrg|newbalanceOrig|isFraud|
+-------+------+-------------+--------------+-------+
|CASH_IN|  1.42|   1270713.41|    1270714.83|      0|
|CASH_IN|  4.35|   4136277.22|    4136281.57|      0|
+-------+------+-------------+--------------+-------+
only showing top 2 rows



### Dtypes
In this dataset, any column of type string is treated as a categorical feature, but sometimes we might have numeric features we want treated as categorical or vice versa. We’ll need to carefully identify which columns are numeric and which are categorical.

In [19]:
train.dtypes

[('type', 'string'),
 ('amount', 'double'),
 ('oldbalanceOrg', 'double'),
 ('newbalanceOrig', 'double'),
 ('isFraud', 'int')]

To use ML we need to conver string into values ,check the Data Type for that!

In [20]:
catCols = [x for (x, dataType) in train.dtypes if dataType == "string"]
numCols = [ x for (x, dataType) in train.dtypes if (dataType == "double") ]
#numCols = [ x for (x, dataType) in train.dtypes if ((dataType == "double") & (x != "isFraud")) ]
#skip the "isFraud" but 

Here we seperated the numerical col and string col;

these fancy is [ ] is just way to get the data in list format!! ie the output will be list!


In [21]:
print(numCols)
print(catCols)

['amount', 'oldbalanceOrg', 'newbalanceOrig']
['type']


### One hot encoding

StringIndexer:
Converts a single feature to an index feature.
http://spark.apache.org/docs/latest/ml-features#stringindexer


OneHotEncoder:
http://spark.apache.org/docs/latest/ml-features#onehotencoder

For more info: http://spark.apache.org/docs/latest/ml-features

In [22]:
train.agg(F.countDistinct("type")).show()

+-----------+
|count(type)|
+-----------+
|          5|
+-----------+



use agg when sum is indeed.
F is sql function ie countDistinct
DF goes along with sql as DF is in tabel format.
SQL function maily applies in the record of the colum or the whole record itself.


In [23]:
train.groupBy("type").count().show()

24/12/12 16:10:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/12 16:10:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/12 16:10:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/12 16:10:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 372791|
| CASH_IN| 979384|
|CASH_OUT|1565928|
| PAYMENT|1504894|
|   DEBIT|  28880|
+--------+-------+



## Encode Categorical Variables using PySpark
Most machine learning algorithms accept the data only in numerical form. So, it is essential to convert any categorical variables present in our dataset into numbers.
* String Indexing is similar to Label Encoding.

## One-Hot Encoding
One-hot encoding is a concept every data scientist should know. I’ve relied on it multiple times when dealing with missing values. It’s a lifesaver!

Here’s the caveat – Spark’s OneHotEncoder does not directly encode the categorical variable.
> First, we need to use the String Indexer to convert the variable into numerical form and then use OneHotEncoderEstimator to encode multiple columns of the dataset.

It creates a Sparse Vector for each row.

In [24]:
from pyspark.ml.feature import (
    OneHotEncoder,
    StringIndexer,
)

What is One Hot Encoding? Why and When Do You Have to Use it?
=> Encode categorical features as a one-hot numeric array. By default, the encoder derives the categories based on the unique values in each feature.

## This is why we use one hot encoder to perform “binarization” of the category and include it as a feature to train the model.

https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f


df = df.select("type","isFraud")
#categorical value 

In [25]:
#catCols are the cols with string
string_indexer = [
    StringIndexer(inputCol=x, outputCol=x + "_StringIndexer", handleInvalid="skip")
    for x in catCols
]

here fun takes 2 parameters and  _StringIndexer will give the numeric value of of x as output when put like this : 
x + "_StringIndexer"

## So we use it to conver string into numerical values.

# fromat for [] is var =[ output function/var for(loop)   ] then the output will stored as a list of item.


In [26]:
string_indexe=string_indexer[0].fit(df).transform(df)
string_indexe.show()

+--------+---------+-------------+--------------+-------+------------------+
|    type|   amount|oldbalanceOrg|newbalanceOrig|isFraud|type_StringIndexer|
+--------+---------+-------------+--------------+-------+------------------+
| PAYMENT|  9839.64|     170136.0|     160296.36|      0|               1.0|
| PAYMENT|  1864.28|      21249.0|      19384.72|      0|               1.0|
|TRANSFER|    181.0|        181.0|           0.0|      1|               3.0|
|CASH_OUT|    181.0|        181.0|           0.0|      1|               0.0|
| PAYMENT| 11668.14|      41554.0|      29885.86|      0|               1.0|
| PAYMENT|  7817.71|      53860.0|      46042.29|      0|               1.0|
| PAYMENT|  7107.77|     183195.0|     176087.23|      0|               1.0|
| PAYMENT|  7861.64|    176087.23|     168225.59|      0|               1.0|
| PAYMENT|  4024.36|       2671.0|           0.0|      0|               1.0|
|   DEBIT|  5337.77|      41720.0|      36382.23|      0|               4.0|

In [27]:
one_hot_encoder = [
    OneHotEncoder(
        inputCols=[f"{x}_StringIndexer" for x in catCols],
        outputCols=[f"{x}_OneHotEncoder" for x in catCols],
    )
]

In [28]:
one_hot_encoder_df=one_hot_encoder[0].fit(string_indexe).transform(string_indexe)
one_hot_encoder_df.show()


+--------+---------+-------------+--------------+-------+------------------+------------------+
|    type|   amount|oldbalanceOrg|newbalanceOrig|isFraud|type_StringIndexer|type_OneHotEncoder|
+--------+---------+-------------+--------------+-------+------------------+------------------+
| PAYMENT|  9839.64|     170136.0|     160296.36|      0|               1.0|     (4,[1],[1.0])|
| PAYMENT|  1864.28|      21249.0|      19384.72|      0|               1.0|     (4,[1],[1.0])|
|TRANSFER|    181.0|        181.0|           0.0|      1|               3.0|     (4,[3],[1.0])|
|CASH_OUT|    181.0|        181.0|           0.0|      1|               0.0|     (4,[0],[1.0])|
| PAYMENT| 11668.14|      41554.0|      29885.86|      0|               1.0|     (4,[1],[1.0])|
| PAYMENT|  7817.71|      53860.0|      46042.29|      0|               1.0|     (4,[1],[1.0])|
| PAYMENT|  7107.77|     183195.0|     176087.23|      0|               1.0|     (4,[1],[1.0])|
| PAYMENT|  7861.64|    176087.23|     1

## one-hot numeric array of the features

### Vector assembling

VectorAssembler:
Combines the values of input columns into a single vector.
http://spark.apache.org/docs/latest/ml-features#vectorassembler

It accepts numeric, boolean and vector type columns


In [29]:
from pyspark.ml.feature import VectorAssembler

In [30]:
assemblerInput = [x for x in numCols]
assemblerInput += [f"{x}_OneHotEncoder" for x in catCols]

In [31]:
assemblerInput

['amount', 'oldbalanceOrg', 'newbalanceOrig', 'type_OneHotEncoder']

In [32]:
vector_assembler = VectorAssembler(
    inputCols=assemblerInput, outputCol="VectorAssembler_features"
)

## Transformers and Estimators
As the name suggests, Transformers convert one dataframe into another either by updating the current values of a particular column (like converting categorical columns to numeric) or mapping it to some other values by using a defined logic.

An Estimator implements the fit() method on a dataframe and produces a model. For example, LogisticRegression is an Estimator that trains a classification model when we call the fit() method.




## Now all the feature of record will be represted by the a single vector

In [33]:
stages = []
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]


We need stages and pipeline so that we don't need to apply the indexer and encoder to to

In [34]:
stages

[StringIndexer_9fa08de04396,
 OneHotEncoder_281d94c6869b,
 VectorAssembler_145518cf57fa]

> A pipeline allows us to maintain the data flow of all the relevant transformations that are required to reach the end result.

In [35]:
#%%time
from pyspark.ml import Pipeline

pipeline = Pipeline().setStages(stages)
model = pipeline.fit(train)

pp_df = model.transform(train)

In [36]:
pp_df.select(
    "type", "amount", "oldbalanceOrg", "newbalanceOrig", "VectorAssembler_features",
).show(truncate=False)

+-------+------+-------------+--------------+---------------------------------------------------+
|type   |amount|oldbalanceOrg|newbalanceOrig|VectorAssembler_features                           |
+-------+------+-------------+--------------+---------------------------------------------------+
|CASH_IN|1.42  |1270713.41   |1270714.83    |[1.42,1270713.41,1270714.83,0.0,0.0,1.0,0.0]       |
|CASH_IN|4.35  |4136277.22   |4136281.57    |[4.35,4136277.22,4136281.57,0.0,0.0,1.0,0.0]       |
|CASH_IN|4.71  |50198.0      |50202.71      |[4.71,50198.0,50202.71,0.0,0.0,1.0,0.0]            |
|CASH_IN|5.19  |18104.0      |18109.19      |[5.19,18104.0,18109.19,0.0,0.0,1.0,0.0]            |
|CASH_IN|5.66  |5061561.06   |5061566.72    |[5.66,5061561.06,5061566.72,0.0,0.0,1.0,0.0]       |
|CASH_IN|6.5   |1696433.45   |1696439.95    |[6.5,1696433.45,1696439.95,0.0,0.0,1.0,0.0]        |
|CASH_IN|8.29  |20392.0      |20400.29      |[8.29,20392.0,20400.29,0.0,0.0,1.0,0.0]            |
|CASH_IN|9.02  |2416

In [37]:
pp_df.show()

+-------+------+-------------+--------------+-------+------------------+------------------+------------------------+
|   type|amount|oldbalanceOrg|newbalanceOrig|isFraud|type_StringIndexer|type_OneHotEncoder|VectorAssembler_features|
+-------+------+-------------+--------------+-------+------------------+------------------+------------------------+
|CASH_IN|  1.42|   1270713.41|    1270714.83|      0|               2.0|     (4,[2],[1.0])|    [1.42,1270713.41,...|
|CASH_IN|  4.35|   4136277.22|    4136281.57|      0|               2.0|     (4,[2],[1.0])|    [4.35,4136277.22,...|
|CASH_IN|  4.71|      50198.0|      50202.71|      0|               2.0|     (4,[2],[1.0])|    [4.71,50198.0,502...|
|CASH_IN|  5.19|      18104.0|      18109.19|      0|               2.0|     (4,[2],[1.0])|    [5.19,18104.0,181...|
|CASH_IN|  5.66|   5061561.06|    5061566.72|      0|               2.0|     (4,[2],[1.0])|    [5.66,5061561.06,...|
|CASH_IN|   6.5|   1696433.45|    1696439.95|      0|           

In [38]:
test.count()

1910743

In [39]:
df_test=test.where(test.isFraud == 1)

In [40]:
df_test.show()

+--------+--------+-------------+--------------+-------+
|    type|  amount|oldbalanceOrg|newbalanceOrig|isFraud|
+--------+--------+-------------+--------------+-------+
|CASH_OUT| 1996.17|      1996.17|           0.0|      1|
|CASH_OUT|  2007.0|       2007.0|           0.0|      1|
|CASH_OUT|  2100.0|       2100.0|           0.0|      1|
|CASH_OUT| 4094.07|      4094.07|           0.0|      1|
|CASH_OUT| 4120.14|      4120.14|           0.0|      1|
|CASH_OUT| 4289.18|      4289.18|           0.0|      1|
|CASH_OUT| 4530.71|      4530.71|           0.0|      1|
|CASH_OUT| 6783.47|      6783.47|           0.0|      1|
|CASH_OUT| 6844.73|      6844.73|           0.0|      1|
|CASH_OUT| 7887.75|      7887.75|           0.0|      1|
|CASH_OUT| 7927.06|      7927.06|           0.0|      1|
|CASH_OUT| 8380.79|      8380.79|           0.0|      1|
|CASH_OUT|  8677.0|       8677.0|           0.0|      1|
|CASH_OUT| 9917.27|      9917.27|           0.0|      1|
|CASH_OUT| 10565.0|      10565.

### Logistic Regression

In [41]:
from pyspark.ml.classification import LogisticRegression

In [42]:
data = pp_df.select(
    F.col("VectorAssembler_features").alias("features"),
    F.col("isFraud").alias("label"),
)

In [43]:
data.show(5, truncate=False)

+--------------------------------------------+-----+
|features                                    |label|
+--------------------------------------------+-----+
|[1.42,1270713.41,1270714.83,0.0,0.0,1.0,0.0]|0    |
|[4.35,4136277.22,4136281.57,0.0,0.0,1.0,0.0]|0    |
|[4.71,50198.0,50202.71,0.0,0.0,1.0,0.0]     |0    |
|[5.19,18104.0,18109.19,0.0,0.0,1.0,0.0]     |0    |
|[5.66,5061561.06,5061566.72,0.0,0.0,1.0,0.0]|0    |
+--------------------------------------------+-----+
only showing top 5 rows



In [44]:
%%time
model = LogisticRegression().fit(data)
data=model.transform(data)


24/12/12 16:12:25 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


CPU times: user 52 ms, sys: 17.5 ms, total: 69.5 ms
Wall time: 1min 7s


In [45]:
data.show()

+--------------------+-----+--------------------+-----------+----------+
|            features|label|       rawPrediction|probability|prediction|
+--------------------+-----+--------------------+-----------+----------+
|[1.42,1270713.41,...|    0|[197.561087692828...|  [1.0,0.0]|       0.0|
|[4.35,4136277.22,...|    0|[203.627061733642...|  [1.0,0.0]|       0.0|
|[4.71,50198.0,502...|    0|[194.977674963424...|  [1.0,0.0]|       0.0|
|[5.19,18104.0,181...|    0|[194.909763566973...|  [1.0,0.0]|       0.0|
|[5.66,5061561.06,...|    0|[205.585769589594...|  [1.0,0.0]|       0.0|
|[6.5,1696433.45,1...|    0|[198.462517102935...|  [1.0,0.0]|       0.0|
|[8.29,20392.0,204...|    0|[194.914769091310...|  [1.0,0.0]|       0.0|
|[9.02,2416260.59,...|    0|[199.986377907442...|  [1.0,0.0]|       0.0|
|[12.18,299322.0,2...|    0|[195.505411293429...|  [1.0,0.0]|       0.0|
|[13.2,106204.0,10...|    0|[195.096672834586...|  [1.0,0.0]|       0.0|
|[14.36,613030.46,...|    0|[196.169582960009...|  

## Model Testing

**Preparing the test data by passing it to the pipeline**

In [46]:
model = pipeline.fit(df_test)

pp_df_test = model.transform(df_test)

In [47]:
data_test = pp_df_test.select(
    F.col("VectorAssembler_features").alias("features"),
    F.col("isFraud").alias("label"),
)

In [48]:
data_test.show(5, truncate=False)

+-------------------------+-----+
|features                 |label|
+-------------------------+-----+
|[1996.17,1996.17,0.0,0.0]|1    |
|[2007.0,2007.0,0.0,0.0]  |1    |
|[2100.0,2100.0,0.0,0.0]  |1    |
|[4094.07,4094.07,0.0,0.0]|1    |
|[4120.14,4120.14,0.0,0.0]|1    |
+-------------------------+-----+
only showing top 5 rows



In [49]:
model = LogisticRegression().fit(data_test)
data=model.transform(data_test)
data.show()

24/12/12 16:13:55 WARN Instrumentation: [be3e43e3] All labels are the same value and fitIntercept=true, so the coefficients will be zeros. Training is not needed.


+--------------------+-----+--------------------+-----------+----------+
|            features|label|       rawPrediction|probability|prediction|
+--------------------+-----+--------------------+-----------+----------+
|[1996.17,1996.17,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[2007.0,2007.0,0....|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[2100.0,2100.0,0....|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[4094.07,4094.07,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[4120.14,4120.14,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[4289.18,4289.18,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[4530.71,4530.71,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[6783.47,6783.47,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[6844.73,6844.73,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[7887.75,7887.75,...|    1|[-Infinity,Infinity]|  [0.0,1.0]|       1.0|
|[7927.06,7927.06,...|    1|[-Infinity,Infinity]|  

In [50]:
df.limit

<bound method DataFrame.limit of DataFrame[type: string, amount: double, oldbalanceOrg: double, newbalanceOrig: double, isFraud: int]>

In [51]:
model.summary.areaUnderROC

1.0

In [52]:
model.summary.pr.show()

+------+---------+
|recall|precision|
+------+---------+
|   0.0|      1.0|
|   1.0|      1.0|
+------+---------+

